In [28]:
import torch
import clip
from PIL import Image


In [29]:
torch.cuda.is_available() 

True

In [30]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [31]:
model, preprocess = clip.load("ViT-B/32", device=device)

In [32]:
print(model)

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [33]:
image = Image.open("../CLIP.png")
print(type(image))
image = preprocess(image)
print(type(image), image.size(), image.device)
image = image.unsqueeze(0)
image.size()
image = image.to(device)
print(image.device)


<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'torch.Tensor'> torch.Size([3, 224, 224]) cpu
cuda:0


In [34]:
text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)

In [35]:
with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu()

In [40]:
with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    logits_per_image, logits_per_text = model(torch.cat([image, image], dim=0), text)
    probs = logits_per_image.softmax(dim=-1).cpu()

In [41]:
print(logits_per_image)
print(logits_per_text)
print(probs)

tensor([[25.5469, 20.0938, 19.7656],
        [25.5469, 20.0938, 19.7656]], device='cuda:0', dtype=torch.float16)
tensor([[25.5469, 25.5469],
        [20.0938, 20.0938],
        [19.7656, 19.7656]], device='cuda:0', dtype=torch.float16)
tensor([[0.9927, 0.0043, 0.0031],
        [0.9927, 0.0043, 0.0031]], dtype=torch.float16)


### softmax

In [37]:
import numpy as np 
a = np.exp(25.5312 - 25.5312)
print(a)
b = np.exp(20.0938 - 25.5312)
print(b)
c = np.exp(19.7500 - 25.5312)
print(c)
d = a + b + c
print(a / d)
print(b / d)
print(c / d)

1.0
0.0043507806074163475
0.0030850111727327013
0.9926190911214203
0.0043186678922023155
0.003062240986377361


In [38]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']